# BRAIN TUMOR DETECTION

## 1. Defining the Question

### a) Specifying the Question

To detect the presence or absence of brain tumors and accurately classify the type of detected tumor as either a meningioma, glioma or pituitary tumor

### b) Defining the Metric for Success

The model will be considered successful, if it can achieve an accuracy of 90% and above

### c) Understanding the context

Brain tumor is the growth of abnormal cells in the tissues of the brain. Brain tumors can be benign (non cancerous) or malignant (cancerous cells), National Cancer Institute. 
 
When a brain tumor grows large enough such that it presses on surrounding nerves, blood vessels and tissues, it can impair brain functionality whether it is cancerous or not. Only about one-third of brain tumors are cancerous.
Tumors that develop in the brain are called primary tumors. Tumors that spread to the brain after forming in different parts of the body like breast, lung, prostate, large bowel (colon), kidney and skins are called secondary tumors or metastatic tumors. 
 
The most common type of primary tumor is the glioma tumor. About 33% of all brain tumors are gliomas. The tumor originates in the glial cells that surround and support neurons in the brain (Johns Hopkins). Meningiomas are also common, and form 30% of all brain tumors (Johns Hopkins). The tumor arises from the meninges, therefore, technically, it is not a brain tumor but is included in this category as it compresses the adjacent brain, nerves and blood vessels (Mayo Clinic). According to the National Cancer Institute, a majority of the tumors are benign. The final example of a primary tumor is a pituitary tumor. They are abnormal growths that develop in the pituitary gland. The tumor can cause over or underproduction of hormones that regulate important functions of the body. Most pituitary tumors are benign and do not spread to other parts of the body (Mayo Clinic). 
 
In Kenya, in the Global Burden of Diseases Study of 2016 published in the Lancet in 2019, it was estimated that 4.3 people in every 100,000 are diagnosed with brain tumors each year,  Nation Africa.
With a population of 47.5 million at the last Kenyan census, that would give an approximate number of 2,200 people per year who get a new diagnosis of a brain tumor, or 42 people per week or 6 people per day.


### d). Recording the Experimental Design

- Data sourcing/loading
- Data Understanding
- Data Relevance
- External Dataset Validation
- Data Preparation
- Descriptive analysis
- Modeling: neural networks
- Implementation of the solution

### e) Data Relevance

The dataset to use for this project can be found by following this link:
 https://www.kaggle.com/code/jaykumar1607/brain-tumor-mri-classification-tensorflow-cnn/data


Assumptions

We shall assume the data is accurate



## 2. Reading the Data

## 3. Checking the Data

## 4. Exploratory Data Analysis (EDA)

## 5. Implementing the Solution